In [1]:
import time
import pandas as pd
import numpy as np
from pycaret.classification import *
import warnings

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("data/train.csv")
train

,individualnumber,category_number,hakkedis_amt,odul_amt,response
0,94230288,9000,21.0,2.0,0
1,4684087,9000,17.0,1.0,0
2,92472145,9058,24.0,3.0,0
3,88026681,9030,22.0,2.0,0
4,98127795,9001,38.0,3.0,0
...,...,...,...,...,...
13110,97214433,9059,27.0,2.0,0
13111,96841665,9004,65.0,6.0,0
13112,98445787,9044,33.0,3.0,0
13113,13781030,9004,66.0,6.0,0


In [3]:
test = pd.read_csv("data/test.csv")
test

,individualnumber,category_number,hakkedis_amt,odul_amt
0,95812936,9029,38.0,3.0
1,23929570,9012,28.0,2.0
2,95948115,9019,21.0,2.0
3,100898513,9035,48.0,4.0
4,12769156,9015,26.0,3.0
...,...,...,...,...
13231,93418529,9049,32.0,3.0
13232,97397069,9018,20.0,2.0
13233,95212479,9046,85.0,8.0
13234,93406449,9032,39.0,3.0


In [4]:
customer = pd.read_csv("data/customer.csv")
customer

,individualnumber,gender,city_code,dateofbirth
0,94212124,K,NaN,1946.0
1,96387515,E,34.0,1974.0
2,95040383,E,80.0,1980.0
3,94694434,E,NaN,1970.0
4,47648671,E,35.0,1963.0
...,...,...,...,...
28588,94368469,K,34.0,1948.0
28589,90691477,E,7.0,1990.0
28590,95834384,E,22.0,1970.0
28591,92199848,E,34.0,1985.0


In [53]:
customeraccount = pd.read_csv("data/customeraccount.csv")
customeraccount

,individualnumber,cardnumber
0,16481068,7287134635560315
1,17634947,6587004647560415
2,19323290,8187034648564315
3,19323290,3005502487247749
4,37820213,6287114610560316
...,...,...
35154,100998656,2687784602565315
35155,101074070,6387894671560235
35156,101270229,9287054697566815
35157,101328888,2587064608564915


In [54]:
transaction_header = pd.read_csv("data/transaction_header.csv")
transaction_header

,date_of_transaction,cardnumber,basketid,is_sanal
0,2020-12-01,8487374630565615,20120163990041163,0
1,2020-12-01,4287574603569015,20120163990016137,0
2,2020-12-01,6487154648560315,20120160080017338,0
3,2020-12-01,3787564693563915,20120163570105022,0
4,2020-12-01,9087994683569715,20120162340016645,0
...,...,...,...,...
1124668,2021-12-01,5187224618568515,21120111950019800,0
1124669,2021-12-01,87714689568015,21120111830079927,0
1124670,2021-12-01,3287454606560415,21120110690047978,0
1124671,2021-12-01,9887854616568815,21120110000068135,0


In [56]:
train_customer = pd.merge(train, customer, how="left")
train_customer_customeraccount = pd.merge(train_customer, customeraccount, how="left")
train_customer_customeraccount_transactionheader =  pd.merge(train_customer_customeraccount, transaction_header, how="left")
train_customer_customeraccount_transactionheader

,individualnumber,category_number,hakkedis_amt,odul_amt,response,gender,city_code,dateofbirth,cardnumber,date_of_transaction,basketid,is_sanal
0,94230288,9000,21.0,2.0,0,E,7.0,1983.0,787714605562415,2020-12-07,2.012076e+16,0.0
1,94230288,9000,21.0,2.0,0,E,7.0,1983.0,787714605562415,2020-12-09,2.012096e+16,0.0
2,94230288,9000,21.0,2.0,0,E,7.0,1983.0,787714605562415,2020-12-10,2.012104e+16,0.0
3,94230288,9000,21.0,2.0,0,E,7.0,1983.0,787714605562415,2020-12-11,2.012114e+16,0.0
4,94230288,9000,21.0,2.0,0,E,7.0,1983.0,787714605562415,2020-12-13,2.012136e+16,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
398455,13781030,9004,66.0,6.0,0,E,54.0,1981.0,6687324629560519,2021-11-12,2.111123e+16,1.0
398456,13781030,9004,66.0,6.0,0,E,54.0,1981.0,6687324629560519,2021-11-13,2.111135e+16,0.0
398457,13781030,9004,66.0,6.0,0,E,54.0,1981.0,6687324629560519,2021-11-24,2.111247e+16,0.0
398458,96703898,9017,15.0,1.0,0,E,34.0,1993.0,1587944647562115,2021-01-12,2.101126e+16,0.0


In [57]:
test_customer = pd.merge(test, customer, how="left")
test_customer_customeraccount = pd.merge(test_customer, customeraccount, how="left")
test_customer_customeraccount_transactionheader =  pd.merge(test_customer_customeraccount, transaction_header, how="left")
test_customer_customeraccount_transactionheader

,individualnumber,category_number,hakkedis_amt,odul_amt,gender,city_code,dateofbirth,cardnumber,date_of_transaction,basketid,is_sanal
0,95812936,9029,38.0,3.0,E,10.0,1995.0,987444691569515,2021-05-17,2.105175e+16,0.0
1,95812936,9029,38.0,3.0,E,10.0,1995.0,987444691569515,2021-06-28,2.106286e+16,0.0
2,95812936,9029,38.0,3.0,E,10.0,1995.0,987444691569515,2021-07-31,2.107316e+16,0.0
3,95812936,9029,38.0,3.0,E,10.0,1995.0,987444691569515,2021-08-01,2.108016e+16,0.0
4,23929570,9012,28.0,2.0,E,34.0,1974.0,5687824656560416,2020-12-20,2.012203e+16,0.0
...,...,...,...,...,...,...,...,...,...,...,...
398016,98792801,9008,20.0,2.0,E,NaN,1980.0,1587664678560719,2021-10-12,2.110124e+16,0.0
398017,98792801,9008,20.0,2.0,E,NaN,1980.0,1587664678560719,2021-10-16,2.110165e+16,0.0
398018,98792801,9008,20.0,2.0,E,NaN,1980.0,1587664678560719,2021-10-23,2.110235e+16,0.0
398019,98792801,9008,20.0,2.0,E,NaN,1980.0,1587664678560719,2021-11-01,2.111014e+16,0.0


In [58]:
train_data = train_customer_customeraccount_transactionheader
test_data = test_customer_customeraccount_transactionheader

In [59]:
s = setup(data = train_data, 
            target = 'response', 
            experiment_name = 'migros',
            fold = 5,
            use_gpu = False,
            normalize = True,
            pca = False,
            remove_outliers = False,
            remove_multicollinearity = True,
            feature_selection = False,
            fix_imbalance = True,
            )

,Description,Value
0,Session id,2216
1,Target,response
2,Target type,Binary
3,Original data shape,"(398460, 12)"
4,Transformed data shape,"(638369, 13)"
5,Transformed train set shape,"(518830, 13)"
6,Transformed test set shape,"(119539, 13)"
7,Numeric features,9
8,Categorical features,2
9,Rows with missing values,26.6%


In [60]:
best_models = compare_models(sort="F1", n_select = 3, fold = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9997,1.0000,0.9967,0.9993,0.9980,0.9978,0.9978,24.2520
et,Extra Trees Classifier,0.9996,0.9999,0.9954,0.9994,0.9974,0.9972,0.9972,27.5480
catboost,CatBoost Classifier,0.9996,0.9997,0.9946,0.9990,0.9968,0.9965,0.9966,57.4580
dt,Decision Tree Classifier,0.9980,0.9972,0.9963,0.9763,0.9862,0.9851,0.9852,3.1980
xgboost,Extreme Gradient Boosting,0.9971,0.9995,0.9932,0.9667,0.9798,0.9782,0.9783,47.1960
lightgbm,Light Gradient Boosting Machine,0.9880,0.9950,0.9497,0.8865,0.9170,0.9105,0.9111,6.1400
knn,K Neighbors Classifier,0.9854,0.9905,0.9780,0.8399,0.9037,0.8958,0.8989,35.8000
gbc,Gradient Boosting Classifier,0.8752,0.9334,0.8463,0.3417,0.4868,0.4300,0.4877,40.9780
ada,Ada Boost Classifier,0.8068,0.8687,0.7596,0.2316,0.3549,0.2774,0.3457,12.1020
ridge,Ridge Classifier,0.6886,0.0000,0.6276,0.1333,0.2199,0.1182,0.1741,1.6640


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

In [61]:
#best_model = best_models[0]
best_model = create_model("lightgbm")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9887,0.9958,0.9564,0.8898,0.9219,0.9158,0.9165
1,0.9875,0.9945,0.9387,0.8888,0.9131,0.9064,0.9068
2,0.9871,0.9957,0.9513,0.8746,0.9113,0.9044,0.9053
3,0.9903,0.9956,0.9567,0.9094,0.9324,0.9272,0.9275
4,0.9863,0.9934,0.9451,0.8700,0.9060,0.8986,0.8995
Mean,0.9880,0.9950,0.9497,0.8865,0.9170,0.9105,0.9111
Std,0.0014,0.0009,0.0069,0.0138,0.0093,0.0100,0.0099


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [62]:
tuned_model = tune_model(best_model, optimize = "F1", n_iter = 2, search_library = 'optuna', choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9920,0.9945,0.9277,0.9572,0.9422,0.9379,0.9381
1,0.9918,0.9940,0.9239,0.9577,0.9405,0.9361,0.9363
2,0.9900,0.9955,0.9354,0.9222,0.9287,0.9233,0.9234
3,0.9924,0.9950,0.9328,0.9569,0.9447,0.9406,0.9407
4,0.9911,0.9932,0.9221,0.9493,0.9355,0.9307,0.9308
Mean,0.9915,0.9944,0.9284,0.9487,0.9383,0.9337,0.9339
Std,0.0009,0.0008,0.0051,0.0136,0.0057,0.0061,0.0062


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2022-12-09 00:12:14,763] Searching the best hyperparameters using 278921 samples...
[I 2022-12-09 00:14:17,421] Finished hyperparemeter search!


In [63]:
# default model
print(best_model)

# tuned model
print(tuned_model)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=2216, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
LGBMClassifier(bagging_fraction=0.43108269066022686, bagging_freq=4,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.6062716726697401, importance_type='split',
               learning_rate=1.2930781077082894e-06, max_depth=-1,
               min_child_samples=37, min_child_weight=0.001,
               min_split_gain=0.7222950751522884, n_estimators=79, n_jobs=-1,
               num_leaves=239, objective=None, random_state=2216,
               reg_alpha=0.7559583974431131

In [64]:
final_model = finalize_model(tuned_model)

In [69]:
prediction_df = predict_model(final_model, data = test_data)

In [90]:
df = prediction_df.groupby(['individualnumber'])['Label'].agg(pd.Series.mode)

0         13146
1            85
[0, 1]        1
[0, 1]        1
[0, 1]        1
[0, 1]        1
[0, 1]        1
Name: Label, dtype: int64

In [110]:
for i in range(len(df)):
    if type(df.iloc[i]) != np.int32:
        df.iloc[i] = df.iloc[i][1]

In [116]:
submission = pd.DataFrame({"individualnumber":test["individualnumber"]})
submission["response"] = list(df)
submission

,individualnumber,response
0,95812936,0
1,23929570,0
2,95948115,0
3,100898513,0
4,12769156,0
...,...,...
13231,93418529,0
13232,97397069,0
13233,95212479,0
13234,93406449,0


In [117]:
submission.to_csv(f"submission/Submission-{time.strftime('%d - %H-%M-%S')}.csv", index=False)